In [1]:
%matplotlib inline

import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from livelossplot import PlotLossesKerasTF
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import util

In [2]:
#tf.debugging.set_log_device_placement(True)
# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
True
Num GPUs Available:  0


In [3]:
folder_path = 'datasets/kaggle_clement/split_files/'
train_file = 'train.csv'
test_file = 'test.csv'
save_file = 'test_pred.csv'

train=pd.read_csv(folder_path+train_file, header=0)
train_X,train_Y=(np.array(train["text"]),np.array(train["label"]))

test=pd.read_csv(folder_path+test_file, header=0)
test_X,test_Y=(np.array(train["text"]),np.array(train["label"]))

In [4]:
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=1000000)
encoder.adapt(train_X)

In [5]:
model=tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary())+1000,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [6]:
plotlosses = PlotLossesKerasTF()

In [ ]:
history=model.fit(train_X[0:1000],train_Y[0:1000],epochs=15,callbacks=[plotlosses],verbose=False, workers=1)

In [8]:
test_loss, test_acc = model.evaluate(test_X[0:10], test_Y[0:10], workers=4)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

10/10 [==============================] - 3s 341ms/sample - loss: 0.6775 - accuracy: 0.6000
Test Loss: 0.6775163412094116
Test Accuracy: 0.6000000238418579
